In [26]:
library(sceasy)
suppressPackageStartupMessages(library(DESeq2))
suppressPackageStartupMessages(library(H5weaver))
suppressPackageStartupMessages(library(dplyr))
suppressPackageStartupMessages(library(tidyr))
suppressPackageStartupMessages(library(tibble))
suppressPackageStartupMessages(library(ggplot2))
suppressPackageStartupMessages(library(Seurat))
suppressPackageStartupMessages(library(stringr))
library(parallel)
library(SingleCellExperiment)
library(knitr)

In [24]:
install.packages('sceasy')

Warning message:
“package ‘sceasy’ is not available for this version of R

A version of this package for your version of R might be available elsewhere,
see the ideas at
https://cran.r-project.org/doc/manuals/r-patched/R-admin.html#Installing-packages”


In [25]:
meta_data=read.csv('/home/jupyter/IH-A-Aging-Analysis-Notebooks/Qiuyu-Notebooks/scRNA/hise_meta_data_2024-01-23_fixed.csv')
head(meta_data)
unique(meta_data$sample.visitName)
length(meta_data$pbmc_sample_id)

,X.1,X,lastUpdated,sample.id,sample.bridgingControl,sample.sampleKitGuid,sample.visitName,sample.visitDetails,sample.drawDate,sample.daysSinceFirstVisit,⋯,pbmc_sample_id,Sample_ID,Sample_Draw_date,Covid_Dose_1,Covid_Dose_2,Covid_exlcusion,CMV.IgG.Serology.Result.Interpretation,BMI,file.name.downloaded,file.path
,<int>,<int>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<int>,⋯,<chr>,<lgl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
1,1,1,2023-11-18T02:03:08.347Z,f499ff83-e513-4d24-a10f-151348269fff,false,KT00001,Flu Year 1 Day 0,N/A - Flu-Series Timepoint Only,2019-10-01T00:00:00Z,0,⋯,PB00001-01,NA,2019-10-15,NA,NA,no,Negative,<25-BMI,B001-P1_PB00001-01_labeled.h5,/home/jupyter/BRI_Analysis/scRNA/cache/fec489f9-9a74-4635-aa91-d2bf09d1faec/B001-P1_PB00001-01_labeled.h5
2,2,2,2023-11-18T02:03:08.347Z,750e90a9-a296-4f0f-969f-60225c2bca17,false,KT00002,Flu Year 1 Day 0,N/A - Flu-Series Timepoint Only,2019-10-01T00:00:00Z,0,⋯,PB00002-01,NA,2019-10-15,2020-12-28,2021-01-18,no,Negative,<25-BMI,B001-P1_PB00002-01_labeled.h5,/home/jupyter/BRI_Analysis/scRNA/cache/7c0c7979-eebd-4aba-b5b2-6e76b4643623/B001-P1_PB00002-01_labeled.h5
3,3,3,2023-11-18T02:03:08.347Z,2db6fb3f-e3f4-454b-891b-9b068541b51d,false,KT00003,Flu Year 1 Day 0,N/A - Flu-Series Timepoint Only,2019-10-01T00:00:00Z,0,⋯,PB00003-01,NA,2019-10-15,2020-12-28,2021-01-16,no,Negative,<25-BMI,B001-P1_PB00003-01_labeled.h5,/home/jupyter/BRI_Analysis/scRNA/cache/40efd03a-cb2f-4677-af42-a056cbfe5a17/B001-P1_PB00003-01_labeled.h5
4,4,4,2023-11-19T10:34:34.12Z,f04693c5-563c-4b5b-ae58-877d0d9ae2fe,false,KT00004,Flu Year 1 Day 0,N/A - Flu-Series Timepoint Only,2019-10-01T00:00:00Z,0,⋯,PB00004-01,NA,2019-10-21,2021-04-16,2021-05-06,no,Negative,<25-BMI,B002-P1_PB00004-01_2023-11-17T21:36:51.794326181Z_labeled.h5,/home/jupyter/BRI_Analysis/scRNA/cache/68fbcd34-1d63-461d-8195-df5b8dc61b31/B002-P1_PB00004-01_2023-11-17T21:36:51.794326181Z_labeled.h5
5,5,5,2023-11-19T05:07:36.805Z,eb5b3a3d-002e-40a6-aa19-aa0e6a7fff8f,false,KT00006,Flu Year 1 Day 0,N/A - Flu-Series Timepoint Only,2019-10-01T00:00:00Z,0,⋯,PB00006-01,NA,2019-10-23,2021-01-16,2021-02-26,no,Negative,<25-BMI,B002-P1_PB00006-01_2023-11-17T21:36:51.794326181Z_labeled.h5,/home/jupyter/BRI_Analysis/scRNA/cache/ea8d98e9-e99e-4dc6-9e78-9866e0deac68/B002-P1_PB00006-01_2023-11-17T21:36:51.794326181Z_labeled.h5
6,6,6,2023-11-18T02:03:08.347Z,f32ac061-0ede-4040-8d8c-35229ea5229c,false,KT00010,Flu Year 1 Day 0,N/A - Flu-Series Timepoint Only,2019-10-01T00:00:00Z,0,⋯,PB00010-02,NA,2019-10-28,2021-03-21,2021-04-18,no,Negative,>25-30BMI,B078-P2_PB00010-02_2021-08-19T17:09:29.934849811Z_labeled.h5,/home/jupyter/BRI_Analysis/scRNA/cache/237e41df-5616-4852-8a8f-b32835c9efcb/B078-P2_PB00010-02_2021-08-19T17:09:29.934849811Z_labeled.h5


[1] "Flu Year 1 Day 0"        "Flu Year 1 Day 7"       
 [3] "Flu Year 1 Day 90"       "Immune Variation Day 0" 
 [5] "Immune Variation Day 7"  "Immune Variation Day 90"
 [7] "Flu Year 2 Stand-Alone"  "Flu Year 1 Stand-Alone" 
 [9] "Flu Year 2 Day 0"        "Flu Year 2 Day 7"       
[11] "Flu Year 2 Day 90"       "Flu Year 3 Stand-Alone"

[1] 868

In [12]:
meta_data_subset <- meta_data %>%
  dplyr::filter(sample.visitName %in% c('Flu Year 1 Day 0') &
         Covid_exlcusion %in% c('no'))
#head(meta_data_subset)
#length(meta_data_subset$sample.visitDetails)
colnames(meta_data_subset)

[1] "X.1"                                   
 [2] "X"                                     
 [3] "lastUpdated"                           
 [4] "sample.id"                             
 [5] "sample.bridgingControl"                
 [6] "sample.sampleKitGuid"                  
 [7] "sample.visitName"                      
 [8] "sample.visitDetails"                   
 [9] "sample.drawDate"                       
[10] "sample.daysSinceFirstVisit"            
[11] "file.id"                               
[12] "file.name"                             
[13] "file.batchID"                          
[14] "file.panel"                            
[15] "file.pool"                             
[16] "file.fileType"                         
[17] "file.majorVersion"                     
[18] "subject.id"                            
[19] "subject.biologicalSex"                 
[20] "subject.birthYear"                     
[21] "subject.ethnicity"                     
[22] "subject.partnerCode"                   
[23] "subject.race"                          
[24] "subject.subjectGuid"                   
[25] "cohort.cohortGuid"                     
[26] "sample.diseaseStatesRecordedAtVisit"   
[27] "pbmc_sample_id"                        
[28] "Sample_ID"                             
[29] "Sample_Draw_date"                      
[30] "Covid_Dose_1"                          
[31] "Covid_Dose_2"                          
[32] "Covid_exlcusion"                       
[33] "CMV.IgG.Serology.Result.Interpretation"
[34] "BMI"                                   
[35] "file.name.downloaded"                  
[36] "file.path"

In [14]:
t(table(meta_data_subset$subject.subjectGuid,meta_data_subset$sample.visitName))

                  
                   BR1001 BR1002 BR1003 BR1004 BR1005 BR1006 BR1007 BR1008
  Flu Year 1 Day 0      1      1      1      1      1      1      1      1
                  
                   BR1009 BR1010 BR1011 BR1012 BR1013 BR1014 BR1015 BR1016
  Flu Year 1 Day 0      1      1      1      1      1      1      1      1
                  
                   BR1017 BR1018 BR1019 BR1021 BR1023 BR1024 BR1025 BR1026
  Flu Year 1 Day 0      1      1      1      1      1      1      1      1
                  
                   BR1028 BR1030 BR1031 BR1032 BR1033 BR1037 BR1040 BR1041
  Flu Year 1 Day 0      1      1      1      1      1      1      1      1
                  
                   BR1042 BR1043 BR1044 BR1045 BR1047 BR1048 BR1049 BR1050
  Flu Year 1 Day 0      1      1      1      1      1      1      1      1
                  
                   BR1051 BR1052 BR1054 BR1056 BR1057 BR1058 BR1059 BR2002
  Flu Year 1 Day 0      1      1      1      1      1      1 

# Creating a psuedobulk object

In [20]:
library(parallel)

bulk_list<-mclapply(meta_data_subset$pbmc_sample_id,function(x){

pbmc <-sceasy::convertFormat(paste0('/home/jupyter/BRI_Analysis/h5_cleaned_by_sample/',x,'.h5ad'), from="anndata", to="seurat")
bulk <- Seurat:::PseudobulkExpression(object = pbmc, pb.method = 'aggregate',   group.by = "AIFI_L3",slot = 'counts')$RNA
colnames(bulk)<-paste0(x,":",colnames(bulk))
return(bulk)
},mc.cores=60)


In [29]:
length(bulk_list)

[1] 92

In [21]:
dim(bulk_list)

NULL

In [34]:
write.csv(bulk_list, file = "Flu_Yr1_D0vsD7_Pseudobulk.csv", row.names = TRUE)

In [22]:
bulk_list=read.csv('Flu_Yr1_D0vsD7_Pseudobulk.csv')

In [23]:
dim(bulk_list)

[1] 33538  6288

In [15]:
head(bulk_csv)

,X,PB00001.01.ASDC,PB00001.01.Activated.memory.B.cell,PB00001.01.Adaptive.NK.cell,PB00001.01.BaEoMaP.cell,PB00001.01.C1Q..CD16.monocyte,PB00001.01.CD4.MAIT,PB00001.01.CD8.MAIT,PB00001.01.CD8aa,PB00001.01.CD14..cDC2,⋯,PB01446.01.Platelet,PB01446.01.Proliferating.NK.cell,PB01446.01.Proliferating.T.cell,PB01446.01.SOX4..Vd1.gdT,PB01446.01.SOX4..naive.CD4.T.cell,PB01446.01.SOX4..naive.CD8.T.cell,PB01446.01.Transitional.B.cell,PB01446.01.Type.2.polarized.memory.B.cell,PB01446.01.cDC1,PB01446.01.pDC
,<chr>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,⋯,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>
1,MIR1302-2HG,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
2,FAM138A,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
3,OR4F5,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
4,AL627309.1,0,0,0,0,0,0,4,0,0,⋯,0,0,0,0,0,0,1,0,0,1
5,AL627309.3,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
6,AL627309.2,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0


In [8]:
for (celltype in str_extract(colnames(bulk_list[[1]]), "(?<=:).*")){
print(celltype)
if (celltype=='Erythrocyte'){next}
matirx_list<-mclapply(bulk_list,function(bulk_matrix){

matching_columns <- grepl(celltype, colnames(bulk_matrix),, fixed = TRUE)
matching_columns<-colnames(bulk_matrix)[matching_columns]
bulk_matrix_single <- as.data.frame(bulk_matrix[, matching_columns, drop = FALSE])
return(bulk_matrix_single)
},mc.cores=60)

df_all<-do.call(cbind,matirx_list)
df_meta_data<-as.data.frame(colnames(df_all))
colnames(df_meta_data)<-'sample_name'
split_columns <- strsplit(df_meta_data$sample_name, ":")
df_meta_data$pbmc_sample_id <- sapply(split_columns, `[`, 1)
df_meta_data$AIFI_L3 <- sapply(split_columns, `[`, 2)
df_meta_data<-left_join(df_meta_data,meta_data_subset)
rownames(df_meta_data)<-df_meta_data$sample_name
selected_gene_list<-read.csv('filtered_gene_list.csv')
selGenes<-selected_gene_list %>% filter(AIFI_L3==celltype) %>% select(gene) %>% pull()  

dds <- DESeqDataSetFromMatrix(df_all[selGenes,], colData = df_meta_data, 
                              design = ~  sample.visitName+
                                          subject.biologicalSex+
                                          CMV.IgG.Serology.Result.Interpretation)
dds <- DESeq(dds,parallel=FALSE)

res <- data.frame(results(dds,contrast=c("sample.visitName", "Flu Year 1 Day 0", "Flu Year 1 Day 7")))
res$celltype<-celltype
res=res %>% arrange(padj) %>% mutate(Direction=case_when(log2FoldChange<0~'HigherInD7',log2FoldChange>0~'HigherInD0'))
res$Year<-"Y1"
write.csv(res,paste0("01C_DESeq2_D0_VS_D7_L3/",celltype,"_Y1.csv"))

}


In [19]:
for (celltype in str_extract(colnames(bulk_list[[1]]), "(?<=:).*")) {
  print(celltype)
  if (celltype == 'Erythrocyte') {
    next
  }
  matirx_list <- mclapply(bulk_list, function(bulk_matrix) {
    matching_columns <- grepl(celltype, colnames(bulk_matrix), fixed = TRUE)
    matching_columns <- colnames(bulk_matrix)[matching_columns]
    bulk_matrix_single <- as.data.frame(bulk_matrix[, matching_columns, drop = FALSE])
    # Write data frame to CSV file
    filename <- paste0(celltype, ".csv")
    write.csv(bulk_matrix_single, file = filename, row.names = FALSE)
    return(bulk_matrix_single)
  }, mc.cores = 60)
}

## Trying Singlecell experiment

In [18]:
sce <- SingleCellExperiment(assays = list(counts = counts), 
                           colData = bulk_list)

ERROR: Error in validObject(assays): invalid class “SimpleAssays” object: 
    'dim(x)' must return a numeric vector
